# Experimenting with NBA API

In [1]:
from nba_api.stats.endpoints import playercareerstats

## Official Stats

In [2]:
# Nikola Jokić
career = playercareerstats.PlayerCareerStats(player_id='203999') 

# pandas data frames (optional: pip install pandas)
career.get_data_frames()[0]

# json
career.get_json()

# dictionary
career.get_dict()

{'resource': 'playercareerstats',
 'parameters': {'PerMode': 'Totals', 'PlayerID': 203999, 'LeagueID': None},
 'resultSets': [{'name': 'SeasonTotalsRegularSeason',
   'headers': ['PLAYER_ID',
    'SEASON_ID',
    'LEAGUE_ID',
    'TEAM_ID',
    'TEAM_ABBREVIATION',
    'PLAYER_AGE',
    'GP',
    'GS',
    'MIN',
    'FGM',
    'FGA',
    'FG_PCT',
    'FG3M',
    'FG3A',
    'FG3_PCT',
    'FTM',
    'FTA',
    'FT_PCT',
    'OREB',
    'DREB',
    'REB',
    'AST',
    'STL',
    'BLK',
    'TOV',
    'PF',
    'PTS'],
   'rowSet': [[203999,
     '2015-16',
     '00',
     1610612743,
     'DEN',
     21.0,
     80,
     55,
     1733.0,
     307,
     600,
     0.512,
     28,
     84,
     0.333,
     154,
     190,
     0.811,
     181,
     379,
     560,
     189,
     79,
     50,
     104,
     208,
     796],
    [203999,
     '2016-17',
     '00',
     1610612743,
     'DEN',
     22.0,
     73,
     59,
     2038.0,
     494,
     854,
     0.578,
     45,
     139,
     0.

## Live Data

In [3]:
from nba_api.live.nba.endpoints import scoreboard

# Today's Score Board
games = scoreboard.ScoreBoard()

# json
games.get_json()

# dictionary
games.get_dict()

{'meta': {'version': 1,
  'request': 'https://nba-prod-us-east-1-mediaops-stats.s3.amazonaws.com/NBA/liveData/scoreboard/todaysScoreboard_00.json',
  'time': '2024-05-01 12:46:04.464',
  'code': 200},
 'scoreboard': {'gameDate': '2024-05-01',
  'leagueId': '00',
  'leagueName': 'National Basketball Association',
  'games': [{'gameId': '0042300105',
    'gameCode': '20240501/MIABOS',
    'gameStatus': 1,
    'gameStatusText': '7:30 pm ET',
    'period': 0,
    'gameClock': '',
    'gameTimeUTC': '2024-05-01T23:30:00Z',
    'gameEt': '2024-05-01T19:30:00Z',
    'regulationPeriods': 4,
    'ifNecessary': False,
    'seriesGameNumber': 'Game 5',
    'gameLabel': 'East - First Round',
    'gameSubLabel': 'Game 5',
    'seriesText': 'BOS leads 3-1',
    'seriesConference': 'East',
    'poRoundDesc': 'First Round',
    'gameSubtype': '',
    'homeTeam': {'teamId': 1610612738,
     'teamName': 'Celtics',
     'teamCity': 'Boston',
     'teamTricode': 'BOS',
     'wins': 3,
     'losses': 1,
  

### Pulling Player Data

In [3]:
from nba_api.stats.endpoints import playercareerstats

# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id="203076")
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,...,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,...,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,...,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,...,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,...,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,...,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,...,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,27.0,62,62,2131.0,551,...,0.846,142,435,577,200,91,143,154,156,1618
8,203076,2020-21,00,1610612747,LAL,28.0,36,36,1162.0,301,...,0.738,62,224,286,110,45,59,74,60,786
9,203076,2021-22,00,1610612747,LAL,29.0,40,40,1404.0,370,...,0.713,106,288,394,122,49,90,82,97,927


In [4]:
from nba_api.stats.static import teams

# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = teams.get_teams()
print("Number of teams fetched: {}".format(len(nba_teams)))
nba_teams[:3]

Number of teams fetched: 30


[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Georgia',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970}]

In [5]:
from nba_api.stats.static import players

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print("Number of players fetched: {}".format(len(nba_players)))
nba_players[:5]

Number of players fetched: 4900


[{'id': 76001,
  'full_name': 'Alaa Abdelnaby',
  'first_name': 'Alaa',
  'last_name': 'Abdelnaby',
  'is_active': False},
 {'id': 76002,
  'full_name': 'Zaid Abdul-Aziz',
  'first_name': 'Zaid',
  'last_name': 'Abdul-Aziz',
  'is_active': False},
 {'id': 76003,
  'full_name': 'Kareem Abdul-Jabbar',
  'first_name': 'Kareem',
  'last_name': 'Abdul-Jabbar',
  'is_active': False},
 {'id': 51,
  'full_name': 'Mahmoud Abdul-Rauf',
  'first_name': 'Mahmoud',
  'last_name': 'Abdul-Rauf',
  'is_active': False},
 {'id': 1505,
  'full_name': 'Tariq Abdul-Wahad',
  'first_name': 'Tariq',
  'last_name': 'Abdul-Wahad',
  'is_active': False}]

In [8]:
spurs = [team for team in nba_teams if team["full_name"] == "San Antonio Spurs"][0]
spurs

{'id': 1610612759,
 'full_name': 'San Antonio Spurs',
 'abbreviation': 'SAS',
 'nickname': 'Spurs',
 'city': 'San Antonio',
 'state': 'Texas',
 'year_founded': 1976}

In [7]:
anthony_edwards = [
    player for player in nba_players if player["full_name"] == "Anthony Edwards"
][0]

rudy_gobert = [
    player for player in nba_players if player["full_name"] == "Rudy Gobert"
][0]

In [13]:
ant_stats = playercareerstats.PlayerCareerStats(player_id="1630162")
ant_playoffs = ant_stats.get_data_frames()[2]

In [11]:
ant_playoffs.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [18]:
ant_career = playercareerstats.PlayerCareerStats(player_id="1630162")
ant_career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630162,2020-21,00,1610612750,MIN,19.0,72,55,2314.0,505,...,0.776,59,277,336,211,82,36,160,129,1392
1,1630162,2021-22,00,1610612750,MIN,20.0,72,72,2466.0,549,...,0.786,62,281,343,275,105,46,190,164,1533
2,1630162,2022-23,00,1610612750,MIN,21.0,79,79,2841.0,707,...,0.756,47,411,458,350,125,58,259,186,1946
3,1630162,2023-24,00,1610612750,MIN,22.0,79,78,2770.0,718,...,0.836,52,378,430,405,101,42,241,141,2049


In [15]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# Get the team ID for the Minnesota Timberwolves
timberwolves_id = teams.find_team_by_abbreviation('MIN')['id']

# Define the season to retrieve data for
season = '2023-24'  # 2024 season

# Create a LeagueGameFinder instance for the Timberwolves and the specified season
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=timberwolves_id, season_nullable=season)

# Get the game logs as a DataFrame
games_df = gamefinder.get_data_frames()[0]

# Filter the DataFrame to include only the Timberwolves' games
timberwolves_games_df = games_df[games_df['TEAM_ID'] == timberwolves_id]

# Extract player names and drop duplicates
player_names = timberwolves_games_df['PLAYER_NAME'].str.split('\\').str[0].drop_duplicates()

# Initialize a dictionary to store player starts
player_starts = {}

# Iterate over each player
for player in player_names:
    # Filter the DataFrame for the current player
    player_games_df = timberwolves_games_df[timberwolves_games_df['PLAYER_NAME'].str.contains(player)]
    
    # Count the number of starts for the player
    starts = len(player_games_df)
    
    # Add the player and their starts to the dictionary
    player_starts[player] = starts

# Convert the dictionary to a DataFrame
player_starts_df = pd.DataFrame.from_dict(player_starts, orient='index', columns=['STARTS'])

# Reset the index and rename the index column to 'PLAYER_NAME'
player_starts_df = player_starts_df.reset_index().rename(columns={'index': 'PLAYER_NAME'})

# Print the resulting DataFrame
print(player_starts_df)

KeyError: 'PLAYER_NAME'